In [99]:
import findspark
findspark.init()

In [100]:
from pyspark.sql import SparkSession
# from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [101]:
from datetime import datetime
import requests
import pandas as pd
import json

In [102]:
obs_post_data = pd.read_csv("./관측소 정보.csv")
obs_post_data.head(5)

,Unnamed: 0,obs_post_id,obs_lat,data_type,obs_post_name,obs_lon,obs_object
0,0,DT_0063,35.024178,조위관측소,가덕도,128.810933,"조위,수온,염분,기온,기압,풍속,풍향"
1,1,DT_0032,37.731944,조위관측소,강화대교,126.522222,"조위,기압"
2,2,DT_0031,34.028333,조위관측소,거문도,127.308889,"조위,수온,염분,기온,기압,풍속,풍향"
3,3,DT_0029,34.801389,조위관측소,거제도,128.699167,"조위,수온,염분,기온,기압,풍속,풍향"
4,4,DT_0058,37.560833,조위관측소,경인항,126.601111,"조위,기온,기압,풍속,풍향"


In [103]:
cluster = '118.34.234.79:7077'
spark = SparkSession \
        .builder \
        .master('local[2]') \
        .appName('tempera_obs') \
        .getOrCreate()

In [104]:
import pprint
pprint.pprint(spark.sparkContext.getConf().getAll())
# spark.stop()

[('spark.driver.host', '172.30.1.15'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.app.id', 'local-1645605249127'),
 ('spark.submit.pyFiles', ''),
 ('spark.driver.port', '58546'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'tempera_obs'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.startTime', '1645605248955'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.master', 'local[2]'),
 ('spark.sql.warehouse.dir',
  'file:/Users/mdgome/workspace/study/StudyDataEngineering/playadata/middleproject/spark-warehouse')]


In [105]:
obs_post_list = obs_post_data['obs_post_id'].values.tolist()
obs_post_list[0:5] # 디버깅 용

['DT_0063', 'DT_0032', 'DT_0031', 'DT_0029', 'DT_0058']

In [106]:
service_key="1LJG14JWnAylzKi5GvmuGg=="
now_datetime=datetime.now().strftime('%Y%m%d')
now_datetime # 디버깅 용

'20220224'

In [67]:
for obs in obs_post_list:
    url="http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey="+service_key+"&ObsCode="+obs+"&Date="+now_datetime+"&ResultType=json"
    url1="http://www.khoa.go.kr/api/oceangrid/tideObsAirPres/search.do?ServiceKey="+service_key+"&ObsCode="+obs+"&Date="+now_datetime+"&ResultType=json"
    url2="http://www.khoa.go.kr/api/oceangrid/tideObsWind/search.do?ServiceKey="+service_key+"&ObsCode="+obs+"&Date="+now_datetime+"&ResultType=json"
    url3="http://www.khoa.go.kr/api/oceangrid/tideObsSalt/search.do?ServiceKey="+service_key+"&ObsCode="+obs+"&Date="+now_datetime+"&ResultType=json"
    response = requests.get(url).json()
    response1 = requests.get(url1).json()
    response2 = requests.get(url2).json()
    response3 = requests.get(url3).json()
    df = spark.createDataFrame(response["result"]["data"])
    df1 = spark.createDataFrame(response1["result"]["data"])
    df2 = spark.createDataFrame(response2["result"]["data"])
    df3 = spark.createDataFrame(response3["result"]["data"])
    
    output_df = df.join(df1,'record_time')#.select(df1.record_time, df1.air_pres, df.air_temp)
    output_df = output_df.join(df2, 'record_time')#.select(output_df.record_time,output_df.air_pres,output_df.air_temp,df2.wind_dir,df2.wind_speed)
    output_df = output_df.join(df3, 'record_time')#.select(output_df.record_time,output_df.air_pres,output_df.air_temp,output_df.wind_dir,output_df.wind_speed,df3.salinity)
    output_df = output_df.withColumn('id',lit(obs))
    # MySQL Data 적재 하는 부분 생성
    

SyntaxError: invalid syntax (3413026616.py, line 3)

In [132]:
url="http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0063&Date=20220217&ResultType=json"
url1="http://www.khoa.go.kr/api/oceangrid/tideObsAirPres/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0063&Date=20220217&ResultType=json"
url2="http://www.khoa.go.kr/api/oceangrid/tideObsWind/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0063&Date=20220217&ResultType=json"
url3="http://www.khoa.go.kr/api/oceangrid/tideObsSalt/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0001&Date=20220217&ResultType=json"

In [133]:
response = requests.get(url).json()
response1 = requests.get(url1).json()
response2 = requests.get(url2).json()
response3 = requests.get(url3).json()

In [115]:
url4="http://www.khoa.go.kr/api/oceangrid/fcServiceInfo/search.do?ServiceKey=wldhxng34hkddbsgm81lwldhxng34hkddbsgm81l==&ResultType=json"
response4 = requests.get(url4).json()

In [116]:
response4["result"]

{'surfing': {'name': '서핑',
  'data': [{'lon': '129.199',
    'area': '남해',
    'name': '송정해수욕장',
    'addr': '',
    'lat': '35.178'}]},
 'ship': {'name': '뱃멀미',
  'data': [{'lon': '125.2812',
    'area': '황해중부',
    'name': '인천-백령',
    'lat': '37.359564'},
   {'lon': '126.049074',
    'area': '황해중부',
    'name': '인천-연평',
    'lat': '37.518449'}]},
 'searoad': {'name': '바다갈라짐',
  'data': [{'lon': '126.3980896',
    'area': '서해',
    'name': '실미도',
    'addr': '인천시 중구 무의동',
    'lat': '37.4025254'},
   {'lon': '126.1597056',
    'area': '서해',
    'name': '소야도',
    'addr': '옹진군 덕적면 소야도리',
    'lat': '37.22381667'}]},
 'scuba': {'name': '스킨스쿠버',
  'data': [{'lon': '128.6136',
    'area': '동해',
    'name': '동명항',
    'addr': '강원도 속초시',
    'lat': '38.2097'},
   {'lon': '128.7951',
    'area': '동해해',
    'name': '남애항',
    'addr': '강원도 양양군 현남면',
    'lat': '37.9442'}]},
 'mudflat': {'name': '갯벌체험',
  'data': [{'lon': '127.036275',
    'area': '남해',
    'name': '수문마을',
    'addr': '전라남도 장흥

In [89]:
type(response)

dict

In [90]:
response["result"]["data"]
# response["result"]["meta"]

[{'record_time': '2022-02-17 00:00:00', 'air_temp': '-1.9'},
 {'record_time': '2022-02-17 00:01:00', 'air_temp': '-2.1'},
 {'record_time': '2022-02-17 00:02:00', 'air_temp': '-1.9'},
 {'record_time': '2022-02-17 00:03:00', 'air_temp': '-2.2'},
 {'record_time': '2022-02-17 00:04:00', 'air_temp': '-2.2'},
 {'record_time': '2022-02-17 00:05:00', 'air_temp': '-2.4'},
 {'record_time': '2022-02-17 00:06:00', 'air_temp': '-2.2'},
 {'record_time': '2022-02-17 00:07:00', 'air_temp': '-2.0'},
 {'record_time': '2022-02-17 00:08:00', 'air_temp': '-1.9'},
 {'record_time': '2022-02-17 00:09:00', 'air_temp': '-2.0'},
 {'record_time': '2022-02-17 00:10:00', 'air_temp': '-2.0'},
 {'record_time': '2022-02-17 00:11:00', 'air_temp': '-2.0'},
 {'record_time': '2022-02-17 00:12:00', 'air_temp': '-2.3'},
 {'record_time': '2022-02-17 00:13:00', 'air_temp': '-2.4'},
 {'record_time': '2022-02-17 00:14:00', 'air_temp': '-2.4'},
 {'record_time': '2022-02-17 00:15:00', 'air_temp': '-2.4'},
 {'record_time': '2022-0

In [23]:
response1["result"]["data"]

[{'record_time': '2016-01-01 00:01:00', 'air_pres': '1030.1'},
 {'record_time': '2016-01-01 00:03:00', 'air_pres': '1030.1'},
 {'record_time': '2016-01-01 00:04:00', 'air_pres': '1030.1'},
 {'record_time': '2016-01-01 00:05:00', 'air_pres': '1030.2'},
 {'record_time': '2016-01-01 00:06:00', 'air_pres': '1030.2'},
 {'record_time': '2016-01-01 00:07:00', 'air_pres': '1030.4'},
 {'record_time': '2016-01-01 00:08:00', 'air_pres': '1030.4'},
 {'record_time': '2016-01-01 00:09:00', 'air_pres': '1030.4'},
 {'record_time': '2016-01-01 00:10:00', 'air_pres': '1030.4'},
 {'record_time': '2016-01-01 00:11:00', 'air_pres': '1030.5'},
 {'record_time': '2016-01-01 00:12:00', 'air_pres': '1030.6'},
 {'record_time': '2016-01-01 00:13:00', 'air_pres': '1030.4'},
 {'record_time': '2016-01-01 00:14:00', 'air_pres': '1030.5'},
 {'record_time': '2016-01-01 00:15:00', 'air_pres': '1030.6'},
 {'record_time': '2016-01-01 00:16:00', 'air_pres': '1030.8'},
 {'record_time': '2016-01-01 00:17:00', 'air_pres': '10

In [ ]:
response2["result"]["data"]

In [134]:
df = spark.createDataFrame(response["result"]["data"])
df1 = spark.createDataFrame(response1["result"]["data"])
df2 = spark.createDataFrame(response2["result"]["data"])
df3 = spark.createDataFrame(response3["result"]["data"])

In [114]:
df4 = spark.createDataFrame(response4["result"])

KeyError: 'data'

In [64]:
# rdd = spark.sparkContext.parallelize([response])

# df = spark.read.option("multiLine","true").json(rdd)

In [135]:
df.show(5)

+--------+-------------------+
|air_temp|        record_time|
+--------+-------------------+
|    -1.9|2022-02-17 00:00:00|
|    -2.1|2022-02-17 00:01:00|
|    -1.9|2022-02-17 00:02:00|
|    -2.2|2022-02-17 00:03:00|
|    -2.2|2022-02-17 00:04:00|
+--------+-------------------+
only showing top 5 rows



In [136]:
df1.show(5)

+--------+-------------------+
|air_pres|        record_time|
+--------+-------------------+
|  1021.9|2022-02-17 00:00:00|
|  1021.9|2022-02-17 00:01:00|
|  1021.8|2022-02-17 00:02:00|
|  1021.9|2022-02-17 00:03:00|
|  1021.9|2022-02-17 00:04:00|
+--------+-------------------+
only showing top 5 rows



In [137]:
df2.show(5)

+-------------------+--------+----------+
|        record_time|wind_dir|wind_speed|
+-------------------+--------+----------+
|2022-02-17 00:00:00|   321.0|       9.0|
|2022-02-17 00:01:00|   320.0|       8.2|
|2022-02-17 00:02:00|   319.0|      12.5|
|2022-02-17 00:03:00|   323.0|       9.3|
|2022-02-17 00:04:00|   333.0|       9.2|
+-------------------+--------+----------+
only showing top 5 rows



In [138]:
df3.show(5)

+-------------------+--------+
|        record_time|salinity|
+-------------------+--------+
|2022-02-17 00:00:00|    30.2|
|2022-02-17 00:01:00|    30.3|
|2022-02-17 00:02:00|    30.2|
|2022-02-17 00:03:00|    30.2|
|2022-02-17 00:04:00|    30.2|
+-------------------+--------+
only showing top 5 rows



In [145]:
output_df = df.join(df1,'record_time')#.select(df1.record_time, df1.air_pres, df.air_temp)
output_df = output_df.join(df2, 'record_time')#.select(output_df.record_time,output_df.air_pres,output_df.air_temp,df2.wind_dir,df2.wind_speed)
output_df = output_df.join(df3, 'record_time')#.select(output_df.record_time,output_df.air_pres,output_df.air_temp,output_df.wind_dir,output_df.wind_speed,df3.salinity)

In [146]:
output_df.show(10)

+-------------------+--------+--------+--------+----------+--------+
|        record_time|air_temp|air_pres|wind_dir|wind_speed|salinity|
+-------------------+--------+--------+--------+----------+--------+
|2022-02-17 07:14:00|    -5.1|  1024.6|   342.0|       5.7|    30.3|
|2022-02-17 07:47:00|    -5.2|  1025.2|   349.0|       5.7|    30.3|
|2022-02-17 08:04:00|    -4.9|  1025.4|   345.0|       5.9|    30.2|
|2022-02-17 00:11:00|    -2.0|  1021.6|   291.0|       9.7|    30.2|
|2022-02-17 08:03:00|    -5.0|  1025.4|   334.0|       5.7|    30.3|
|2022-02-17 02:19:00|    -3.1|  1022.3|   339.0|       7.2|    30.2|
|2022-02-17 02:26:00|    -3.2|  1022.3|   342.0|       8.4|    30.2|
|2022-02-17 09:39:00|    -4.6|  1025.7|   316.0|       5.9|    30.3|
|2022-02-17 10:07:00|    -4.2|    1026|   320.0|       4.4|    30.2|
|2022-02-17 11:57:00|    -2.8|  1025.4|   309.0|       3.6|    30.2|
+-------------------+--------+--------+--------+----------+--------+
only showing top 10 rows



In [ ]:
df.createOrReplaceTempView('a')
df1.createOrReplaceTempView('b')

spark.sql("SELECT a.record_time, a.air_temp, b.air_pres from a as a, b as b").show()

In [147]:
output_df = output_df.withColumn('id',lit(response["result"]["meta"]["obs_post_id"]))

In [149]:
output_df.show(5)

+-------------------+--------+--------+--------+----------+--------+-------+
|        record_time|air_temp|air_pres|wind_dir|wind_speed|salinity|     id|
+-------------------+--------+--------+--------+----------+--------+-------+
|2022-02-17 07:14:00|    -5.1|  1024.6|   342.0|       5.7|    30.3|DT_0063|
|2022-02-17 07:47:00|    -5.2|  1025.2|   349.0|       5.7|    30.3|DT_0063|
|2022-02-17 08:04:00|    -4.9|  1025.4|   345.0|       5.9|    30.2|DT_0063|
|2022-02-17 00:11:00|    -2.0|  1021.6|   291.0|       9.7|    30.2|DT_0063|
|2022-02-17 08:03:00|    -5.0|  1025.4|   334.0|       5.7|    30.3|DT_0063|
+-------------------+--------+--------+--------+----------+--------+-------+
only showing top 5 rows



In [151]:
ip='220.78.231.223'

In [152]:
output_df.write.format('jdbc').options(
    url='jdbc:mysql://'+ip+'/ocean_'+now_datetime,
    driver='com.mysql.jdbc.Driver',
    dbtable=response["result"]["meta"]["obs_post_id"],
    user='root',
    password='Rlawjdals1!').mode('append').save()

Py4JJavaError: An error occurred while calling o874.save.
: java.lang.ClassNotFoundException: com.mysql.jdbc.Driver
	at java.net.URLClassLoader.findClass(URLClassLoader.java:382)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:101)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:218)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:222)
	at org.apache.spark.sql.execution.d

In [153]:
spark.stop()